In [121]:
import pandas
import requests
import datetime
from bokeh.charts import Bar, output_file, show, TimeSeries, Scatter

output_notebook()

Loading BokehJS ...

In [122]:
def get_indicator(countries, indicator):
    url = 'http://api.worldbank.org/countries/%s/indicators/%s?per_page=300&date=2014:2014&format=json' % (';'.join(countries), ';'.join(indicator))
    assert isinstance(countries, list)
    assert isinstance(indicator, list)
    response = requests.get(url)
    json = response.json()
    dataset = json[1]
    data = {
        'Date': [x['date'] for x in dataset],
        'Country': [x['country']['value'] for x in dataset],
        'Indicator': [x['indicator']['value'] for x in dataset],
        'Value': [x['value'] for x in dataset],
    }
    df = pandas.DataFrame(data)
    df['Value'] = pandas.to_numeric(df['Value'], errors='coerce')
    df['Date'] = pandas.to_datetime(df['Date'])
    df = df.round(2)
    return df

In [123]:
countries = ['BRA', 'USA', 'VEN', 'GBR']
indicator = ['CC.EST']
df = get_indicator(countries,indicator)
df

,Country,Date,Indicator,Value
0,Brazil,2014-01-01,Control of Corruption: Estimate,-0.38
1,United Kingdom,2014-01-01,Control of Corruption: Estimate,1.73
2,United States,2014-01-01,Control of Corruption: Estimate,1.32
3,"Venezuela, RB",2014-01-01,Control of Corruption: Estimate,-1.38


In [124]:
def plot_ind(performance):
    p = Bar(performance, label='Country', values='Value', color='Country', xlabel="Country", 
            ylabel=performance['Indicator'][0], title='Performance of %s' % performance['Indicator'][0], legend='top_right')
    return p

In [125]:
plot = plot_ind(df)
show(plot)

In [126]:
def create_dataframes(countries, indicators):
    dataframes = [get_indicator(countries, i) for i in indicators]
    return pandas.concat(dataframes)

In [127]:
countries = ['BRA', 'USA']
i1 = ['CC.EST']
i2 = ['RL.EST']
indicators = [i1, i2]
df = create_dataframes(countries,indicators)
df

,Country,Date,Indicator,Value
0,Brazil,2014-01-01,Control of Corruption: Estimate,-0.38
1,United States,2014-01-01,Control of Corruption: Estimate,1.32
0,Brazil,2014-01-01,Rule of Law: Estimate,-0.08
1,United States,2014-01-01,Rule of Law: Estimate,1.62


In [128]:
def get_mean(dataframes):
    score = df.groupby('Country')['Value'].mean().reset_index()
    return score

In [129]:
score = get_mean(df)
score

,Country,Value
0,Brazil,-0.23
1,United States,1.47


In [130]:
def plot_score(score):
    p = Bar(score, label='Country', values='Value', xlabel="Country", 
    color='Country', title='Title', legend='top_right')
    return p

In [131]:
plotscore = plot_score(score)
show(plotscore)